In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import cartopy
import dask
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
import fsspec
import requests
import aiohttp
import nc_time_axis
import cftime

/var/folders/7y/lr_w90f56s35gmw3l6ghkz_w0000gn/T/ipykernel_4308/1470668176.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

In [5]:
var_df = pd.read_csv("Financial_Support_Agreement_NEW.csv")
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [ ]:
# df_ta_ncar = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2' & variable_id == 'tas'  & institution_id == 'NCAR'")
df_ta_ncar = df.query("experiment_id == 'historical' & table_id =='Amon' & source_id == 'CESM2' & activity_id=='CMIP' & institution_id == 'NCAR'")
vars_here = df_ta_ncar["variable_id"].copy()
# df_ta_ncar.experiment_id.unique()

#display(df_ta_ncar)

# display(vars_here)
display(var_df[var_df["variable_id"].isin(vars_here)])

,variable_id,description,table_id(s)
58,cfc11global,Global Mean Mole Fraction of CFC11 [1e-12],Amon
60,cfc12global,Global Mean Mole Fraction of CFC12 [1e-12],Amon
63,ch4global,Global Mean Mole Fraction of CH4 [1e-09],Amon
81,ci,Fraction of Time Convection Occurs in Cell [1],"Amon, CF3hr, CFsubhr"
97,cli,Mass Fraction of Cloud Ice [kg kg-1],"Amon, CFsubhr"
110,clivi,Ice Water Path [kg m-2],"Amon, CF3hr, CFday, CFsubhr"
114,cl,Percentage Cloud Cover [%],"Amon, CFday, CFsubhr"
120,clt,Total Cloud Cover Percentage [%],"3hr, Amon, CF3hr, CFsubhr, day"
122,clw,Mass Fraction of Cloud Liquid Water [kg kg-1],"Amon, CFsubhr"
126,clwvi,Condensed Water Path [kg m-2],"Amon, CFsubhr"


In [10]:
# df_ta_ncar = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2' & variable_id == 'tas'  & institution_id == 'NCAR'")
df_ta_ncar = df.query("experiment_id == 'historical' & table_id == 'Amon' &  source_id == 'CESM2' & activity_id=='CMIP'")
vars_here = df_ta_ncar["variable_id"].copy()
# df_ta_ncar.experiment_id.unique()
display(df_ta_ncar.head())
print(var_df[var_df["variable_id"].isin(vars_here)][["description", "variable_id"]])

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
59364,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,ch4global,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59373,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,cfc11global,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59374,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,cfc12global,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59382,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59383,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,rldscs,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308


                                            description  variable_id
58           Global Mean Mole Fraction of CFC11 [1e-12]  cfc11global
60           Global Mean Mole Fraction of CFC12 [1e-12]  cfc12global
63             Global Mean Mole Fraction of CH4 [1e-09]    ch4global
81       Fraction of Time Convection Occurs in Cell [1]           ci
97                 Mass Fraction of Cloud Ice [kg kg-1]          cli
110                             Ice Water Path [kg m-2]        clivi
114                          Percentage Cloud Cover [%]           cl
120                    Total Cloud Cover Percentage [%]          clt
122       Mass Fraction of Cloud Liquid Water [kg kg-1]          clw
126                       Condensed Water Path [kg m-2]        clwvi
131                  Total Atmospheric Mass of CO2 [kg]      co2mass
132                    Mole Fraction of CO2 [mol mol-1]          co2
250   Evaporation Including Sublimation and Transpir...      evspsbl
371             Surface Upward Lat